In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This script is used to aggregate samples from different chunks of mEBC data when SCRaPL is fitted with inflation. Results from this script are used to estimate DIC for the model with inflation.

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import scipy
import scipy.stats

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from tensorflow import keras

from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import pickle
from timeit import default_timer as timer
Folder = '/content/drive/MyDrive/'

In [ ]:
#For each one of the chunks store parameters of interest in a list.
cor_nuts_col = []
s_acc_nuts_col = []
s_exp_nuts_col = []
m_acc_nuts_col = []
m_exp_nuts_col = []
inf_acc_nuts_col = []
inf_exp_nuts_col = []
llk_nuts_col = []
x_acc_mn_nuts_col = []
x_exp_mn_nuts_col = []
aaa = 1416
for prt in range(aaa):
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_cor_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        cor_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_s_acc_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        s_acc_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_s_exp_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        s_exp_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_m_acc_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        m_acc_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_m_exp_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        m_exp_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_inf_acc_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        inf_acc_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_inf_exp_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        inf_exp_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/log_prob_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        llk_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/avg_exp_inf_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        x_exp_mn_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_atac_DIC/avg_met_inf_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
        x_acc_mn_nuts_col.append(pickle.load(handle))

    print(prt/aaa)

0.0
0.0007062146892655367
0.0014124293785310734
0.00211864406779661
0.002824858757062147
0.0035310734463276836
0.00423728813559322
0.0049435028248587575
0.005649717514124294
0.006355932203389831
0.007062146892655367
0.007768361581920904
0.00847457627118644
0.009180790960451978
0.009887005649717515
0.01059322033898305
0.011299435028248588
0.012005649717514125
0.012711864406779662
0.013418079096045197
0.014124293785310734
0.014830508474576272
0.015536723163841809
0.016242937853107344
0.01694915254237288
0.01765536723163842
0.018361581920903956
0.019067796610169493
0.01977401129943503
0.020480225988700564
0.0211864406779661
0.021892655367231638
0.022598870056497175
0.023305084745762712
0.02401129943502825
0.024717514124293787
0.025423728813559324
0.026129943502824857
0.026836158192090395
0.02754237288135593
0.02824858757062147
0.028954802259887006
0.029661016949152543
0.03036723163841808
0.031073446327683617
0.03177966101694915
0.03248587570621469
0.033192090395480225
0.03389830508474576


In [ ]:
prt = aaa
with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_cor_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    cor_nuts_col.append(temp[:,tf.newaxis])
with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_s_acc_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    s_acc_nuts_col.append(temp[:,tf.newaxis])
with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_s_exp_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    s_exp_nuts_col.append(temp[:,tf.newaxis])
with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_m_acc_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    m_acc_nuts_col.append(temp[:,tf.newaxis])
with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_m_exp_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    m_exp_nuts_col.append(temp[:,tf.newaxis])
with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_inf_acc_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    inf_acc_nuts_col.append(temp[:,tf.newaxis])
with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_inf_exp_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    temp = pickle.load(handle)
    inf_exp_nuts_col.append(temp[:,tf.newaxis])
with open(Folder+'SCRaPL/Real/Results_atac_DIC/log_prob_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    llk_nuts_col.append(pickle.load(handle))
with open(Folder+'SCRaPL/Real/Results_atac_DIC/avg_exp_inf_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    x_exp_mn_nuts_col.append(pickle.load(handle))
with open(Folder+'SCRaPL/Real/Results_atac_DIC/avg_met_inf_atac_'+str(prt+1)+'.pickle', 'rb') as handle:
    x_acc_mn_nuts_col.append(pickle.load(handle))

In [ ]:
#Concat all lists to aggregate all samples.
cor_nuts = tf.concat(cor_nuts_col,axis=-1)
s_acc_nuts = tf.concat(s_acc_nuts_col,axis=-1)
s_exp_nuts = tf.concat(s_exp_nuts_col,axis=-1)
m_acc_nuts = tf.concat(m_acc_nuts_col,axis=-1)
m_exp_nuts = tf.concat(m_exp_nuts_col,axis=-1)
inf_acc_nuts = tf.concat(inf_acc_nuts_col,axis=-1)
inf_exp_nuts = tf.concat(inf_exp_nuts_col,axis=-1)
llk_nuts = tf.concat(llk_nuts_col,axis=-1)
x_exp_mn_nuts = tf.concat(x_exp_mn_nuts_col,axis=-2)
x_acc_mn_nuts = tf.concat(x_acc_mn_nuts_col,axis=-2)

In [ ]:
#Save parameters of interest.
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_cor_atac_dic.pickle', 'wb') as handle:
    pickle.dump(cor_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_m_acc_atac_dic.pickle', 'wb') as handle:
    pickle.dump(m_acc_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_m_exp_atac_dic.pickle', 'wb') as handle:
    pickle.dump(m_exp_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_s_acc_atac_dic.pickle', 'wb') as handle:
    pickle.dump(s_acc_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_s_exp_atac_dic.pickle', 'wb') as handle:
    pickle.dump(s_exp_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_inf_acc_atac_dic.pickle', 'wb') as handle:
    pickle.dump(inf_acc_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_inf_exp_atac_dic.pickle', 'wb') as handle:
    pickle.dump(inf_exp_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_llk_atac_dic.pickle', 'wb') as handle:
    pickle.dump(llk_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_x_acc_mn_avg_dic.pickle', 'wb') as handle:
    pickle.dump(x_acc_mn_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_atac_DIC_col/nuts_x_exp_mn_avg_dic.pickle', 'wb') as handle:
    pickle.dump(x_exp_mn_nuts, handle)
